<a href="https://colab.research.google.com/github/lordkevinmo/bertelsmann-tech-scholarship-challenge/blob/master/u_net_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def imshow(img):
  img = img/2 + 0.5
  plt.show(np.transpose(img, (1,2,0)))

In [0]:
""" 
  Double conv 2D in UNET architecture.
  Each DOuble Conv represents :
  2 x Conv -> Batch_normalization -> ReLU
"""
class DoubleConv2D(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(DoubleConv2D, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )
  
  def forward(self,x):
    return self.conv(x)


In [0]:
""""
  Downscaling with maxpool of kernel size (2,2)
  and followed by double conv
"""
class DownScaling(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(DownScaling, self).__init__()
    self.pooling_double_conv = nn.Sequential(
        nn.MaxPool2d(2),
        DoubleConv2D(in_channels, out_channels)
    )

  def forward(self, x):
    return self.pooling_double_conv(x)